## Setup & Imports

Importiere alle benötigten Bibliotheken für den Workflow.

In [1]:
# Basic imports
import os
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# PDF processing
from pdfminer.high_level import extract_text
import fitz  # PyMuPDF
from pdf2image import convert_from_path
import pytesseract

# Image processing
import cv2

# Vector graphics
import svgwrite

# NLP
import spacy

# Set Tesseract path for Windows
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'

# Display settings
get_ipython().run_line_magic('matplotlib', 'inline')

print("Alle Bibliotheken erfolgreich importiert!")


Alle Bibliotheken erfolgreich importiert!


## 1. Input Verification

Überprüfe, ob die PDF bereits durchsuchbar ist und extrahiere Text mit pdfminer.six oder PyMuPDF.


In [2]:
def extract_text_from_pdf(pdf_path, method='auto'):
    """
    Extrahiert Text aus einer PDF-Datei mit verschiedenen Methoden.
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        method (str): 'auto', 'pdfminer', 'pymupdf' oder 'ocr'
        
    Returns:
        tuple: (extracted_text, method_used)
    """
    
    if not os.path.exists(pdf_path):
        raise FileNotFoundError(f"PDF-Datei nicht gefunden: {pdf_path}")
    
    # Method 1: pdfminer.six
    if method in ['auto', 'pdfminer']:
        try:
            print("Versuche Textextraktion mit pdfminer.six...")
            text = extract_text(pdf_path)
            if text.strip():  # Check if meaningful text was extracted
                print(f"✓ Text erfolgreich extrahiert ({len(text)} Zeichen)")
                return text, 'pdfminer'
            else:
                print("⚠ Kein Text mit pdfminer gefunden")
        except Exception as e:
            print(f"⚠ pdfminer Fehler: {e}")
    
    # Method 2: PyMuPDF
    if method in ['auto', 'pymupdf']:
        try:
            print("Versuche Textextraktion mit PyMuPDF...")
            doc = fitz.open(pdf_path)
            text = ""
            for page_num in range(len(doc)):
                page = doc[page_num]
                text += page.get_text()
            doc.close()
            
            if text.strip():
                print(f"✓ Text erfolgreich extrahiert ({len(text)} Zeichen)")
                return text, 'pymupdf'
            else:
                print("⚠ Kein Text mit PyMuPDF gefunden")
        except Exception as e:
            print(f"⚠ PyMuPDF Fehler: {e}")
    
    # Method 3: OCR fallback
    if method in ['auto', 'ocr']:
        print("Fallback: OCR wird verwendet...")
        return extract_text_with_ocr(pdf_path), 'ocr'
    
    return "", 'none'

# Test der Funktion (mit Platzhalter)
text, method = extract_text_from_pdf("Werkvertrag Stahlzargentueren - Theodor Christen.pdf")
print(f"Verwendete Methode: {method}")
print(f"Extrahierter Text (erste 200 Zeichen): {text[:200]}...")


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, def

Versuche Textextraktion mit pdfminer.six...
⚠ Kein Text mit pdfminer gefunden
Versuche Textextraktion mit PyMuPDF...
⚠ Kein Text mit PyMuPDF gefunden
Fallback: OCR wird verwendet...


NameError: name 'extract_text_with_ocr' is not defined

## 2. OCR mit Tesseract

Konvertiere PDF-Seiten zu Bildern und extrahiere Text mit Tesseract OCR.


In [3]:
def extract_text_with_ocr(pdf_path, language='deu+eng'):
    """
    Extrahiert Text aus PDF mit OCR (Tesseract).
    
    Args:
        pdf_path (str): Pfad zur PDF-Datei
        language (str): Tesseract Sprachcode (deu=Deutsch, eng=Englisch)
        
    Returns:
        str: Extrahierter Text
    """
    
    try:
        print("Konvertiere PDF zu Bildern...")
        # PDF zu Bildern konvertieren
        images = convert_from_path(pdf_path, dpi=300)  # Höhere DPI für bessere OCR
        
        extracted_text = ""
        
        print(f"Verarbeite {len(images)} Seiten mit OCR...")
        for i, image in enumerate(images):
            print(f"  Seite {i+1}/{len(images)}")
            
            # OCR auf jedes Bild anwenden
            page_text = pytesseract.image_to_string(
                image, 
                lang=language,
                config='--psm 1'  # Page segmentation mode
            )
            
            extracted_text += f"\n--- Seite {i+1} ---\n{page_text}\n"
        
        print(f"✓ OCR abgeschlossen ({len(extracted_text)} Zeichen extrahiert)")
        return extracted_text
        
    except Exception as e:
        print(f"❌ OCR Fehler: {e}")
        return ""


In [4]:
def preprocess_image_for_ocr(image):
    """
    Verbessert ein Bild für bessere OCR-Ergebnisse.
    
    Args:
        image: PIL Image oder numpy array
        
    Returns:
        Processed image
    """
    
    # Konvertiere zu numpy array falls nötig
    if hasattr(image, 'mode'):
        image = np.array(image)
    
    # Zu Graustufen konvertieren
    if len(image.shape) == 3:
        gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
    else:
        gray = image
    
    # Rauschen reduzieren
    denoised = cv2.medianBlur(gray, 3)
    
    # Kontrast verbessern
    clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
    enhanced = clahe.apply(denoised)
    
    return enhanced

# Test der OCR-Funktion (mit Platzhalter)
ocr_text = extract_text_with_ocr("Werkvertrag Stahlzargentueren - Theodor Christen.pdf")
print(f"OCR Text (erste 200 Zeichen): {ocr_text[:200]}...")


Konvertiere PDF zu Bildern...
Verarbeite 23 Seiten mit OCR...
  Seite 1/23
  Seite 2/23
  Seite 3/23
  Seite 4/23
  Seite 5/23
  Seite 6/23
  Seite 7/23
  Seite 8/23
  Seite 9/23
  Seite 10/23
  Seite 11/23
  Seite 12/23
  Seite 13/23
  Seite 14/23
  Seite 15/23
  Seite 16/23
  Seite 17/23
  Seite 18/23
  Seite 19/23
  Seite 20/23
  Seite 21/23
  Seite 22/23
  Seite 23/23
✓ OCR abgeschlossen (50037 Zeichen extrahiert)
OCR Text (erste 200 Zeichen): 
--- Seite 1 ---
cas [ It trl] bauplanungen gmbh

Werkvertrag Nr. 18

Projekt: 2022-1

Dersbachstrasse 307-313, 6333 Hünenberg See
Objekt: Dersbachstrasse 313
Bauherr: Unternehmer:
Theodor Christen He...
